# Projeto de Férias - Aprendizado de Máquina
-----------------------

# Predição de emissão de gases em um dataset sobre turbinas 

**Aluno:** Enzo J. Xavier - RM 24035

**Orientador:** Dr. Daniel Roberto Cassar

## Introdução:

Relembrar: Explicar notebook anterior, referenciar link 

Explicar: Falar objetivo do notebook atual, passo a passo

Modelar: Abordar os modelos usados, teoria e fórmula

## Importações e definições:

Importando as principais bibliotecas usadas neste notebok. A documentação de cada biblioteca se encontra no final do arquivo, na sessão "Referências"

In [1]:
import os
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

import lightning as L
import darts


In [3]:
TAMANHO_LOTE = 128

class TempDataset(Dataset):
    def __init__(self, sequencias):
        self.sequencias = sequencias

    def __len__(self):
        return len(self.sequencias)

    def __getitem__(self, idx):
        sequencia, target = self.sequencias[idx]
        return torch.FloatTensor(sequencia), torch.FloatTensor(target)

In [9]:
class DataModule(L.LightningDataModule):
    def __init__(self):
        super().__init__()

    def setup(self, stage):
        """Aqui devemos alterar o estado da classe para adicionar as informações 
        referentes aos conjuntos de treino, teste e validação. O argumento `stage` 
        deve existir e ele indica em qual estágio o processo de treino está 
        (pode ser `fit` para treinamento/validação e `test` para teste).

        É nesta etapa onde aplicamos transformações aos dados caso necessário."""

        # Diretório
        atual = os.getcwd()
        caminho = os.path.dirname(atual)
        os.chdir(f'{caminho}\\Datasets')
        
        # Definições
        ATRIBUTOS = ['AT','AP','AH','AFDP','GTEP','TIT','TAT','TEY','CDP']
        TARGET = ['CO', 'NOX']
        TAMANHO_SEQUENCIA = 20 # a cada 20 horas

        # Carregar datasets
        df_2011 = pd.read_csv('gt_2011.csv')
        df_2012 = pd.read_csv('gt_2012.csv')
        df_2013 = pd.read_csv('gt_2013.csv')
        df_2014 = pd.read_csv('gt_2014.csv')
        df_2015 = pd.read_csv('gt_2015.csv')

        # Dividir datasets
        df_treino = pd.concat([df_2011,df_2012], ignore_index= True)
        df_val = df_2013
        df_teste = pd.concat([df_2014,df_2015], ignore_index= True)
        df_tot = pd.concat([df_2011,df_2012,df_2013,df_2014,df_2015], ignore_index= True)

        # Organizar e normalizar dados
        X_treino = df_treino.reindex(ATRIBUTOS, axis=1).values
        y_treino = df_treino.reindex(TARGET, axis=1).values

        self.x_scaler = StandardScaler()
        self.x_scaler.fit(X_treino)
        #self.y_scaler = StandardScaler() # não vou ajustar pro y - vazar dados. 
                                         # só criar modelo e transformar funciona?

        # Criar batches
        def cria_sequencias(data, tamanho_sequencia):
            '''Divide o dataset em exemplos (batches) para o treinamento do LSTM'''
            sequencias = []

            for i in range(len(data) - tamanho_sequencia):
                sequencia = data[i : i + tamanho_sequencia]
                target = data[i + tamanho_sequencia]
                sequencias.append((sequencia, target))

            return sequencias

        # Controlar modo de operação
        if stage == "fit":
            # Treino
            X_treino_norm = self.x_scaler.transform(X_treino)
            #y_treino_norm = self.y_scaler.transform(y_treino)
            seq_treino = cria_sequencias(X_treino_norm, TAMANHO_SEQUENCIA)

            self.X_treino_norm = torch.tensor(X_treino_norm, dtype=torch.float32)
            self.y_treino = torch.tensor(y_treino, dtype=torch.float32)

            # Validação
            X_val = df_val.reindex(ATRIBUTOS, axis=1).values
            y_val = df_val.reindex(TARGET, axis=1).values
            X_val_norm = self.x_scaler.transform(X_val)
            #y_val_norm = self.y_scaler.transform(y_val)
            seq_val = cria_sequencias(X_val_norm, TAMANHO_SEQUENCIA)

            self.X_val_norm = torch.tensor(X_val_norm, dtype=torch.float32)
            self.y_val = torch.tensor(y_val, dtype=torch.float32)
        if stage == "test":
            # Teste
            X_teste = df_teste.reindex(ATRIBUTOS, axis=1).values
            y_teste = df_teste.reindex(TARGET, axis=1).values

            X_teste_norm = self.x_scaler.transform(X_teste)
            #y_teste_norm = self.y_scaler.transform(y_teste)

            seq_teste = cria_sequencias(X_teste_norm, TAMANHO_SEQUENCIA)

            self.X_teste_norm = torch.tensor(X_teste_norm, dtype=torch.float32)
            self.y_teste = torch.tensor(y_teste, dtype=torch.float32)

    def train_dataloader(self):
        dataset_treino = TempDataset(seq_treino)
        dataloader_treino = DataLoader(
            dataset_treino, batch_size=TAMANHO_LOTE, shuffle=False)
        return dataloader_treino

    def val_dataloader(self):
        dataset_val = TempDataset(seq_val)
        dataloader_val = DataLoader(
            dataset_val, batch_size=TAMANHO_LOTE, shuffle=False)
        return dataloader_val

    def test_dataloader(self):
        dataset_teste = TempDataset(seq_teste)
        dataloader_teste = DataLoader(
            dataset_teste, batch_size=TAMANHO_LOTE, shuffle=False)
        return dataloader_teste

### Darts:

### Pytorch

In [4]:
class TimeSeriesModel(L.LightningModule):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()

        self.lstm = torch.nn.LSTM(num_inputs, num_hidden, batch_first=True)
        self.fc = torch.nn.Linear(num_hidden, num_outputs) # function
        self.loss = F.mse_loss

        # Curva de aprendizado
        self.perdas_treino = []
        self.perdas_val = []

        self.curva_aprendizado_treino = []
        self.curva_aprendizado_val = []

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.01)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss(y, y_pred)

        self.log("train_loss", loss, prog_bar=True)
        self.perdas_treino.append(loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss(y, y_pred)

        self.log("val_loss", loss, prog_bar=True)
        self.perdas_val.append(loss)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss(y, y_pred)

        self.log("test_loss", loss)

        return loss

    def on_train_epoch_end(self):
        # Atualiza curva de aprendizado treino
        perda_media = torch.stack(self.perdas_treino).mean()
        self.curva_aprendizado_treino.append(float(perda_media))
        self.perdas_treino.clear()

    def on_validation_epoch_end(self):
        # Atualiza curva de aprendizado validação
        perda_media = torch.stack(self.perdas_val).mean()
        self.curva_aprendizado_val.append(float(perda_media))
        self.perdas_val.clear()

In [10]:
NUM_EPOCAS = 5
treinador = L.Trainer(max_epochs=NUM_EPOCAS)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
stage = "fit" 
ATRIBUTOS = ['AT']
TARGET = ['CO', 'NOX']


dm = DataModule()
dm.setup('fit')

In [15]:
num_inputs = len(ATRIBUTOS)
num_hidden = 1
num_outputs = len(TARGET)
#taxa_aprendizado = 0.01

modelo = TimeSeriesModel(
    num_inputs, num_hidden, num_outputs
)

In [16]:
treinador.fit(modelo, dm)


  | Name | Type   | Params
--------------------------------
0 | lstm | LSTM   | 16    
1 | fc   | Linear | 4     
--------------------------------
20        Trainable params
0         Non-trainable params
20        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

NameError: name 'seq_val' is not defined

In [ ]:
ca_treino = modelo.curva_aprendizado_treino
ca_val = modelo.curva_aprendizado_val

sns.lineplot(ca_treino, label="Treino")
eixo = sns.lineplot(ca_val, label="Validação")

eixo.set_xlim(left=0)

eixo.set_title("Curva de aprendizado")
eixo.set_xlabel("Época")
eixo.set_ylabel("Loss");

### Conclusão

### Referências

[1]

[2]

[3]

[4]

[] Biblioteca do normalizador padrão - scikit learn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

[] David Menotti, aula sobre MLP: https://www.inf.ufpr.br/menotti/ci171-182/slides/ci171-classMLP.pdf